In [20]:
import numpy as np
import os
import scipy
import matplotlib.pyplot as plt
import sklearn
import librosa
import librosa.display
from dataset import ALCDataset

%matplotlib inline

In [18]:
SR = 16000

### Load data

In [11]:
alc_dataset = ALCDataset('/Users/mazeyu/Desktop/CMU/20fall/18797/project/code/data')
data_train, meta_train = alc_dataset.load_data('train', num_threads=4)
data_dev1, meta_dev1 = alc_dataset.load_data('d1', num_threads=4)
data_dev2, meta_dev2 = alc_dataset.load_data('d2', num_threads=4)
data_test, meta_test = alc_dataset.load_data('test', num_threads=4)

In [36]:
assert len(data_train) == len(meta_train)
assert len(data_dev1) == len(meta_dev1)
assert len(data_dev2) == len(meta_dev2)
assert len(data_test) == len(meta_test)

print('#train: {}'.format(len(data_train)))
print('#dev1: {}'.format(len(data_dev1)))
print('#dev2: {}'.format(len(data_dev2)))
print('#test: {}'.format(len(data_test)))

#train: 5400
#dev1: 3960
#dev2: 1500
#test: 3000


### Feature extraction

In [25]:
class ALCFeature:
    def __init__(self, sr):
        self.sr = sr
        
    def get_mfcc(self, data, n_mfcc=20, scale=False, show=0):
        mfcc = []
        for i, audio in enumerate(data):
            mfcc_ = librosa.feature.mfcc(audio, sr=self.sr, n_mfcc=n_mfcc)
            if scale:
                mfcc_ = sklearn.preprocessing.scale(mfcc_, axis=1)
            if i < show:
                plt.figure()
                librosa.display.specshow(mfcc_, sr=self.sr, x_axis='time')
                plt.show()
            mfcc.append(mfcc_.flatten())
        mfcc = np.stack(mfcc, axis=0)
        return mfcc
    
    def get_cqt(self, data, wind_length, scale=False, show=0):
        cqt = []
        wind = np.hanning(int(wind_length * self.sr))
        for i, audio in enumerate(data):
            cqt_ = librosa.feature.chroma_cqt(audio, sr=self.sr, window=wind)
            if scale:
                cqt_ = sklearn.preprocessing.scale(cqt_, axis=1)
            if i < show:
                plt.figure()
                librosa.display.specshow(cqt_, sr=self.sr, x_axis='time')
                plt.show()
            cqt.append(cqt_.flatten())
        cqt = np.stack(cqt, axis=0)
        return cqt
    
    def pncc(self, data):
        pass

In [28]:
alc_feature = ALCFeature(SR)

x_train = alc_feature.get_mfcc(data_train)
x_dev1 = alc_feature.get_mfcc(data_dev1)
x_dev2 = alc_feature.get_mfcc(data_dev2)
x_test = alc_feature.get_mfcc(data_test)
y_train = meta_train['label']
y_dev1 = meta_dev1['label']
y_dev2 = meta_dev2['label']
y_test = meta_test['label']

### Classification model

In [ ]:
class ALCModel:
    def __init__(self, method):
        if method == 'lr':
            self.clf = sklearn.linear_model.LogisticRegression()
        elif method == 'svm':
            self.clf = sklearn.svm.SVC(C=1.0, kernel='rbf')
        elif method == 'forest':
            self.clf = sklearn.ensemble.RandomForestClassifier(n_estimators=100)
        elif method == 'adaboost':
            self.clf = sklearn.ensemble.AdaBoostClassifier(n_estimators=100)
        else:
            pass
        
    def fit(self, x, y):
        self.clf.fit(x, y)
    
    def predict(self, x):
        pred = self.clf.predict(x)
        return pred
    
    def evaluate(self, x, y)
        pred = self.clf.predict(x)
        acc = clf.score(x, y)
        report = sklearn.metrics.classification_report(y, pred)
        return acc, report